In [ ]:
import os

# Move to Thesis directory (two levels up)
os.chdir(os.path.abspath(os.path.join("..", "..")))

# Move to model/src if it exists
model_dir = os.path.join(os.getcwd(), "model", "src")
if os.path.exists(model_dir):
    os.chdir(model_dir)

print("Current Directory:", os.getcwd())


# Imports

In [2]:
import selenium
import requests
import pandas as pd
import numpy as np 
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import os
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager


## Loading dictionary with all the program links

In [3]:
dict_programs_links = pd.read_pickle(r"../../data/Webscrapping/programs_links.pkl")
dict_programs_links

In [4]:
if len(dict_programs_links) > 1:
    # Convert dictionary items to a list and get the second key-value pair
    items = list(dict_programs_links.items())
    second_url, second_filtered_links = items[1]
    
    # Print the second URL and filtered links
    print(f"Second URL: {second_url}")
    print(f"Filtered links for the second URL: {second_filtered_links}")
elif dict_programs_links:
    print("The dictionary does not have a second item.")
else:
    print("The dictionary is empty.")

## Exploration of the postgraduate and masters webpage design

### Postgraduate

In [ ]:
# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

first_url = second_filtered_links[0]

# Open webpage
driver.get(first_url)
time.sleep(5)  # Wait for the page to load

# Extract the <main> element
main_element = driver.find_element(By.TAG_NAME, "main")

# Get the inner HTML of <main>
main_html = main_element.get_attribute('outerHTML')

# Close the WebDriver
driver.quit()

# Parse with BeautifulSoup
soup = BeautifulSoup(main_html, "html.parser")


# REMOVE THE BROCHURE AND COUNTRIES SECTIONS
unwanted_section = soup.find("div", class_="hero-article__info")
if unwanted_section:
    unwanted_section.decompose()  

# REMOVING THE STUDY PLAN (1), TEACHING STAFF (2), AND ACADEMIC CALENDER (3) SECTIONS
articles = soup.find_all("article", class_="content__article")
indices_to_remove = [0, 1, 2]  # 0-based index for 1th, 2th, and 3th articles
for index in indices_to_remove:
    if index < len(articles):
        articles[index].decompose()
    

# Extract only the text, removing extra whitespace
main_text = soup.get_text(separator="\n", strip=True)

# Print extracted text
print(main_text)


### Master

In [ ]:
# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

first_url = "https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/master-degree-program-in-data-science-and-advanced-analytics-with-a-specialization-in-data-science/"

# Open webpage
driver.get(first_url)
time.sleep(5)  # Wait for the page to load

# Extract the <main> element
main_element = driver.find_element(By.TAG_NAME, "main")

# Get the inner HTML of <main>
main_html = main_element.get_attribute('outerHTML')

# Close the WebDriver
driver.quit()

# Parse with BeautifulSoup
soup = BeautifulSoup(main_html, "html.parser")

# REMOVE THE BROCHURE AND COUNTRIES SECTIONS
unwanted_section = soup.find("div", class_="hero-article__info")
if unwanted_section:
    unwanted_section.decompose()  

# REMOVING THE STUDY PLAN (1), TEACHING STAFF (2), AND ACADEMIC CALENDER (3) SECTIONS
articles = soup.find_all("article", class_="content__article")
indices_to_remove = [0, 1, 2]  # 0-based index for 1th, 2th, and 3th articles
for index in indices_to_remove:
    if index < len(articles):
        articles[index].decompose()

# Extract only the text, removing extra whitespace
main_text = soup.get_text(separator="\n", strip=True)

# Print extracted text
print(main_text)

In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from webdriver_manager.chrome import ChromeDriverManager

# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Target URL
url = "https://www.novaims.unl.pt/pt/ensino/cursos/licenciaturas/ciencia-de-dados/"

# Open webpage
driver.get(url)
time.sleep(5)  # Wait for the page to load

# Extract the <main> element
main_element = driver.find_element(By.TAG_NAME, "main")

# Get the inner HTML of <main>
main_html = main_element.get_attribute('outerHTML')

# Parse the original page with BeautifulSoup
soup = BeautifulSoup(main_html, "html.parser")



# Find specific links
links_to_extract = [
    "https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/master-degree-program-in-data-science-and-advanced-analytics-with-a-specialization-in-data-science/study-plan/",
    "https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/master-degree-program-in-data-science-and-advanced-analytics-with-a-specialization-in-data-science/teaching-staff/"
]

# Extract text from each specified link and save to different files
for link in links_to_extract:
    # Navigate to the new URL
    driver.get(link)
    time.sleep(5)  # Wait for the new page to load

    # Extract the new page's <main> element
    new_main_element = driver.find_element(By.TAG_NAME, "main")
    new_main_html = new_main_element.get_attribute('outerHTML')

    # Parse the new page with BeautifulSoup
    new_soup = BeautifulSoup(new_main_html, "html.parser")

    # Extract only the text from the new page, removing extra whitespace
    new_main_text = new_soup.get_text(separator="\n", strip=True)

    # Print extracted text from the new page
    print(f"\nText from {link}:")
    print(new_main_text)

# Close the WebDriver
driver.quit()

**Conclusions:**
- Both postgraduate and masters webpage design are very similar 

## Pipeline for Postgraduate and Master Degrees

I'm ignoring at the moment schedules and timetables

In [5]:
# Check if the dictionary is not empty
if dict_programs_links:
    # Get the first key (URL) and its value (filtered links)
    first_url, first_filtered_links = next(iter(dict_programs_links.items()))
    
    # Print the first URL and filtered links
    print(f"First URL: {first_url}")
    print(f"Filtered links for the first URL: {first_filtered_links}")

    # Extract the first three elements from the dictionary
    first_three_programs = list(dict_programs_links.items())[:3]

    # Iterate over the first three programs
    for program_url, program_links in first_three_programs:
        # Extract the name of the bachelor degree from the URL
        posgrad_master_degree_name = program_url.split('/')[-2]

        # Substitute the name in the links
        links_to_extract = [
            f"https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/{posgrad_master_degree_name}/program/#",
            f"https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/{posgrad_master_degree_name}/teaching-staff/#"
            
        ]

        # Print the substituted links
        print(f"Substituted links for {posgrad_master_degree_name}: {links_to_extract}")

        # Optionally, interact with the page after it loads
else:
    print("The dictionary is empty.")


## Main code 

In [6]:
import re

In [ ]:
# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Directory for storing extracted text
postgraduates_masters_dir = r"..\..\data\Webscrapping\postgraduate_master_degrees"
os.makedirs(postgraduates_masters_dir, exist_ok=True)  # Ensure directory exists

def sanitize_filename(filename):
    """Sanitize the filename to remove any invalid characters."""
    # Replace invalid characters with underscores
    return re.sub(r'[<>:"/\\|?*]', '_', filename)

def check_page_exists(url):
    """Returns True if the page contains a <main> element, otherwise False."""
    try:
        driver.get(url)
        time.sleep(3)  # Allow page to load
        driver.find_element(By.TAG_NAME, "main")  # Try finding <main>
        return True  # If found, page exists
    except NoSuchElementException:
        return False  # <main> not found, assume page doesn't exist
    except TimeoutException:
        return False  # Page timeout, assume it doesn't exist
    except Exception as e:
        print(f"Error checking {url}: {e}")
        return False  # Catch-all for unexpected errors

def extract_main_text(url, remove_teaching_staff, remove_study_plan):
    """Extracts the main content text from a given URL, adjusting section removals dynamically."""
    driver.get(url)
    time.sleep(5)  # Wait for page load

    try:
        main_element = driver.find_element(By.TAG_NAME, "main")
        main_html = main_element.get_attribute('outerHTML')
        soup = BeautifulSoup(main_html, "html.parser")

        # Adjust which sections to remove
        if not remove_study_plan and not remove_teaching_staff:
            remove_indices = [0, 1, 2]  # Default: remove Study Plan (1), Teaching Staff (2), Academic Calendar (3)
        elif not remove_study_plan:
            remove_indices = [0, 2]  # Remove First Section (0) and Teaching Staff (2)
        else:
            remove_indices = [2]  # Remove only Teaching Staff (2)

        # Remove unwanted sections
        for selector in [
            ("div", "hero-article__info", None),  # Remove Brochure and Country Sections
            ("article", "content__article", remove_indices)
        ]:
            elements = soup.find_all(selector[0], class_=selector[1])
            if selector[2]:  # If we have specific indices to remove
                for index in selector[2]:
                    if index < len(elements):
                        elements[index].decompose()
            elif elements:  # Remove the single found element
                elements[0].decompose()

        return soup.get_text(separator="\n", strip=True)
    except NoSuchElementException:
        print(f"Error extracting text from {url}: <main> element not found.")
        return ""
    except Exception as e:
        print(f"Error extracting text from {url}: {e}")
        return ""

# Process second URL in `dict_programs_links`
if len(dict_programs_links) > 1:
    second_url, second_filtered_links = list(dict_programs_links.items())[1]
    print(f"Processing: {second_url}")

    for program_url in second_filtered_links:
        posgrad_master_degree_name = program_url.split('/')[-2]  # Extract name from URL

        # URLs to check
        teaching_staff_url = f"https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/{posgrad_master_degree_name}/teaching-staff/#"
        study_plan_url = f"https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/{posgrad_master_degree_name}/study-plan/#"

        # Check if these pages exist
        study_plan_exists = check_page_exists(study_plan_url)
        teaching_staff_exists = check_page_exists(teaching_staff_url)

        # Extract and save main course text
        main_text = extract_main_text(program_url, not teaching_staff_exists, not study_plan_exists)
        if main_text:
            # Sanitize posgrad_master_degree_name to ensure it's a valid file name
            sanitized_name = sanitize_filename(posgrad_master_degree_name)
            
            # Construct file path with sanitized name
            file_path = os.path.join(postgraduates_masters_dir, f"{sanitized_name}_main_course_extracted_text.txt")
            
            # Write extracted text to the file
            with open(file_path, "w", encoding="utf-8") as file:
                file.write(f"Text from {program_url}:\n{main_text}\n\n")

        # Extract and save text from study-plan & teaching-staff pages
        for url, exists in [(study_plan_url, study_plan_exists), (teaching_staff_url, teaching_staff_exists)]:
            if exists:
                extracted_text = extract_main_text(url, not teaching_staff_exists, not study_plan_exists)
                if extracted_text:
                    # Sanitize posgrad_master_degree_name and page-specific name
                    sanitized_name = sanitize_filename(posgrad_master_degree_name)
                    page_name = url.split('/')[-2]  # Extract page name (study-plan, teaching-staff)
                    file_name = os.path.join(postgraduates_masters_dir, f"{sanitized_name}_{page_name}_extracted_text.txt")
                    
                    # Write extracted text to the file
                    with open(file_name, "w", encoding="utf-8") as file:
                        file.write(f"Text from {url}:\n{extracted_text}\n\n")

    print("Text extraction completed and saved.")
else:
    print("The dictionary is empty.")

# Close WebDriver
driver.quit()


In [5]:
# Extract course names from dict_programs_links
if len(dict_programs_links) > 1:
    second_url, second_filtered_links = list(dict_programs_links.items())[1]  # Get second key-value pair
    course_names = [url.split('/')[-2] for url in second_filtered_links]  # Extract course names from URLs
else:
    print("Dictionary does not have enough data.")
    course_names = []

# Expected file templates for each course
expected_file_templates = [
    "{}_main_course_extracted_text.txt",
    "{}_study-plan_extracted_text.txt",
    "{}_teaching-staff_extracted_text.txt"
]

# Tracking processed and missing courses
processed_courses = []
missing_files = {}

# Check if each expected file exists
for course in course_names:
    missing_files[course] = []
    for template in expected_file_templates:
        file_name = template.format(course)
        file_path = os.path.join(postgraduates_masters_dir, file_name)

        if os.path.exists(file_path):
            processed_courses.append(file_name)
        else:
            missing_files[course].append(file_name)

# Generate a verification report
report_path = os.path.join(postgraduates_masters_dir, "course_files_verification_report.txt")
with open(report_path, "w", encoding="utf-8") as report_file:
    report_file.write("### Processed Course Files ###\n")
    report_file.write("\n".join(processed_courses) + "\n\n")

    report_file.write("### Missing Course Files ###\n")
    for course, missing in missing_files.items():
        if missing:
            report_file.write(f"{course}:\n" + "\n".join(missing) + "\n\n")

print(f"Verification report saved: {report_path}")


In [7]:
# URL to check
target_url = "https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/master-degree-in-geographic-information-systems-and-science-with-a-specialization-in-geographic-information-systems-and-science/"

# Check if the URL exists in the dictionary
url_found = any(target_url in links for links in dict_programs_links.values())

# Output the result
if url_found:
    print(f"✅ The URL exists in dict_programs_links.")
else:
    print(f"❌ The URL was NOT found in dict_programs_links.")

In [61]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# URL for the Data Science program
data_science_url = "https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/data-science/"

# Open webpage
driver.get(data_science_url)
time.sleep(5)  # Wait for the page to load

# Extract the <main> element
main_element = driver.find_element(By.TAG_NAME, "main")
main_html = main_element.get_attribute('outerHTML')

# Parse the page with BeautifulSoup
soup = BeautifulSoup(main_html, "html.parser")


# Remove unwanted sections
unwanted_sections = soup.find_all("div", class_="block")
if len(unwanted_sections) >= 4:
    unwanted_sections[3].decompose()  # Remove only the fourth occurrence'
    

if len(unwanted_sections) >= 6:
    unwanted_sections[5].decompose()  # Remove the sixth occurrence'
    
if len(unwanted_sections) >= 7:
    unwanted_sections[6].decompose()  # Remove the seventh occurrence


unwanted_section = soup.find("div", class_="hero-article__info")
if unwanted_section:
    unwanted_section.decompose()

# Remove the specified articles
articles = soup.find_all("article", class_="content__article")
indices_to_remove = [5, 7, 8]  # 0-based index for 6th, 8th, and 9th articles
for index in indices_to_remove:
    if index < len(articles):
        articles[index].decompose()

# Extract only the text, removing extra whitespace
main_text = soup.get_text(separator="\n", strip=True)

# Print extracted text
print("Extracted Text from Data Science Main Course Page:")
print(main_text)

# Close the WebDriver
driver.quit()

## FIN


In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from webdriver_manager.chrome import ChromeDriverManager

# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Target URL
url = "https://www.novaims.unl.pt/pt/ensino/cursos/licenciaturas/ciencia-de-dados/"

# Open webpage
driver.get(url)
time.sleep(5)  # Wait for the page to load

# Extract the <main> element
main_element = driver.find_element(By.TAG_NAME, "main")

# Get the inner HTML of <main>
main_html = main_element.get_attribute('outerHTML')

# Close the WebDriver
driver.quit()

# Parse with BeautifulSoup
soup = BeautifulSoup(main_html, "html.parser")

# Extract only the text, removing extra whitespace
main_text = soup.get_text(separator="\n", strip=True)

# Print extracted text
print(main_text)



In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from webdriver_manager.chrome import ChromeDriverManager

# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Target URL
url = "https://www.novaims.unl.pt/pt/ensino/cursos/licenciaturas/ciencia-de-dados/"

# Open webpage
driver.get(url)
time.sleep(5)  # Wait for the page to load

# Extract the <main> element
main_element = driver.find_element(By.TAG_NAME, "main")

# Get the inner HTML of <main>
main_html = main_element.get_attribute('outerHTML')

# Close the WebDriver
driver.quit()

# Parse with BeautifulSoup
soup = BeautifulSoup(main_html, "html.parser")

# Remove the unwanted section
unwanted_section = soup.find("div", class_="hero-article__info")
if unwanted_section:
    unwanted_section.decompose()  # Removes the element from the soup
    

# Extract only the text, removing extra whitespace
main_text = soup.get_text(separator="\n", strip=True)

# Print extracted text
print(main_text)

# Optionally, save to a text file
with open("ciencia_de_dados_filtered_text.txt", "w", encoding="utf-8") as file:
    file.write(main_text)



In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
from webdriver_manager.chrome import ChromeDriverManager

# Setup WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode (no UI)
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# Target URL
url = "https://www.novaims.unl.pt/pt/ensino/cursos/licenciaturas/ciencia-de-dados/"

# Open webpage
driver.get(url)
time.sleep(5)  # Wait for the page to load

# Scroll to ensure content is fully loaded
driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
time.sleep(3)  # Allow content to load

# Extract elements with the desired class
elements = driver.find_elements(By.CLASS_NAME, "slider-quote")

# Get text content from each element
extracted_texts = [element.text.strip() for element in elements if element.text.strip()]

# Close the WebDriver
driver.quit()

# Print extracted text
for text in extracted_texts:
    print(text)




In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from webdriver_manager.chrome import ChromeDriverManager

In [16]:


# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Target URL
url = "https://www.novaims.unl.pt/pt/ensino/cursos/licenciaturas/ciencia-de-dados/"

# Open webpage
driver.get(url)
time.sleep(5)  # Wait for the page to load

# Extract the <main> element
main_element = driver.find_element(By.TAG_NAME, "main")

# Get the inner HTML of <main>
main_html = main_element.get_attribute('outerHTML')

# Parse the original page with BeautifulSoup
soup = BeautifulSoup(main_html, "html.parser")

# Find the button link dynamically
button = soup.find("a", class_="content__link arrow-animate")
if button:
    new_url = button['href']
    if not new_url.startswith("http"):
        new_url = "https://www.novaims.unl.pt" + new_url

    # Navigate to the new URL
    driver.get(new_url)
    time.sleep(5)  # Wait for the new page to load

    # Extract the new page's <main> element
    new_main_element = driver.find_element(By.TAG_NAME, "main")
    new_main_html = new_main_element.get_attribute('outerHTML')

    # Parse the new page with BeautifulSoup
    new_soup = BeautifulSoup(new_main_html, "html.parser")

    # Extract only the text from the new page, removing extra whitespace
    new_main_text = new_soup.get_text(separator="\n", strip=True)

    # Print extracted text from the new page
    print("\nNew Page Text:")
    print(new_main_text)

# Close the WebDriver
driver.quit()

# Remove unwanted sections
unwanted_sections = soup.find_all("div", class_="block")
if len(unwanted_sections) >= 4:
    unwanted_sections[3].decompose()  # Remove only the fourth occurrence
if len(unwanted_sections) >= 6:
    unwanted_sections[5].decompose()  # Remove the sixth occurrence
if len(unwanted_sections) >= 7:
    unwanted_sections[6].decompose()  # Remove the seventh occurrence

unwanted_section = soup.find("div", class_="hero-article__info")
if unwanted_section:
    unwanted_section.decompose()

# Extract only the text, removing extra whitespace
main_text = soup.get_text(separator="\n", strip=True)

# Print extracted text from the original page
print("Original Page Text:")
print(main_text)

# Optionally, save to a text file
with open("ciencia_de_dados_filtered_text_3.txt", "w", encoding="utf-8") as file:
    file.write(main_text)


### getting every link inside div class="block has-aside":

In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from webdriver_manager.chrome import ChromeDriverManager

# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Target URL
url = "https://www.novaims.unl.pt/pt/ensino/cursos/licenciaturas/ciencia-de-dados/"

# Open webpage
driver.get(url)
time.sleep(5)  # Wait for the page to load

# Extract the <main> element
main_element = driver.find_element(By.TAG_NAME, "main")

# Get the inner HTML of <main>
main_html = main_element.get_attribute('outerHTML')

# Parse the original page with BeautifulSoup
soup = BeautifulSoup(main_html, "html.parser")

# Find all links inside the div with class "block has-aside"
block_div = soup.find("div", class_="block has-aside")
links = block_div.find_all("a", href=True)

# Print all found links
for link in links:
    print(link['href'])

# Close the WebDriver
driver.quit()

In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from webdriver_manager.chrome import ChromeDriverManager

# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Target URL
url = "https://www.novaims.unl.pt/pt/ensino/cursos/licenciaturas/ciencia-de-dados/"

# Open webpage
driver.get(url)
time.sleep(5)  # Wait for the page to load

# Extract the <main> element
main_element = driver.find_element(By.TAG_NAME, "main")

# Get the inner HTML of <main>
main_html = main_element.get_attribute('outerHTML')

# Parse the original page with BeautifulSoup
soup = BeautifulSoup(main_html, "html.parser")

# Find specific links
links_to_extract = [
    "https://www.novaims.unl.pt/pt/ensino/cursos/licenciaturas/ciencia-de-dados/plano-de-estudos-ciencia-de-dados/#",
    "https://www.novaims.unl.pt/pt/ensino/cursos/licenciaturas/ciencia-de-dados/corpo-docente/#"
]

# Extract text from each specified link and save to different files
for link in links_to_extract:
    # Navigate to the new URL
    driver.get(link)
    time.sleep(5)  # Wait for the new page to load

    # Extract the new page's <main> element
    new_main_element = driver.find_element(By.TAG_NAME, "main")
    new_main_html = new_main_element.get_attribute('outerHTML')

    # Parse the new page with BeautifulSoup
    new_soup = BeautifulSoup(new_main_html, "html.parser")

    # Extract only the text from the new page, removing extra whitespace
    new_main_text = new_soup.get_text(separator="\n", strip=True)

    # Determine the filename based on the link
    if "plano-de-estudos-ciencia-de-dados" in link:
        filename = "plano_de_estudos_ciencia_de_dados.txt"
    elif "corpo-docente" in link:
        filename = "corpo_docente_ciencia_de_dados.txt"
    else:
        continue

    # Save the text content to a file
    with open(filename, "w", encoding="utf-8") as file:
        file.write(new_main_text)

    # Print extracted text from the new page
    print(f"\nText from {link}:")
    print(new_main_text)

# Close the WebDriver
driver.quit()